# Sanity check
## 1. Cel sprawozdania
Celem tego pliku jest dokumentacja kodu oraz przebadanie poprawności naszej symulacji poprzez porównanie z teoretycznym, spodziewanym wynikiem. 
Na końcu znajdują się wyniki analizy statystycznej po wielokrotnym uruchomieniu symulacji dla prostego przykładu.
## 2. Dokumentacja kodu

In [195]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import pandas as pd
import simpy
import seaborn
import matplotlib.pyplot as plt

Każdy obiekt w naszej symulacji jest klasą, która przechowuje informacje o sobie.

Dla pacjenta są to:
- id: identyfikator Pacjenta
- arrival_time: Godzina, o której przyszedł
- service_start_time: Godzina, o której wszedł do gabinetu (rozpoczęła się obsługa)
- service_end_time: Godzina, o której wyszedł z gabinetu (zakończenie obsługi)

Każda zmienna **time** jest przechowywana jako liczba minut od rozpoczęcia symulacji

In [196]:
class Pacjent:
    def __init__(self, id):
        self.day = 1
        self.id = id
        self.room = None
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None

Dla gabinetu są to:
- id: identyfikator gabinetu
- env: środowisko symulacji
- patients_served: liczba obsłużonych pacjentów

In [197]:
class Gabinet:
    def __init__(self, id, env):
        self.id = id
        self.env = env
        self.patients_served = 0

## Parametry kliniki:
- **`number_of_ rooms`**: liczba gabinetów
- **`service_time`**: Jak długo obsługiwany jest pacjent [minuty]
- **`lambda_per_hour`**: Ile pacjentów przychodzi co godzinę (dzieli się przez 60 bo zamieniamy na minuty)
- **`sim_time`**: Długość symulacji

## Pomocnicze zmienne:
- **`queue_array`**: Kolejka (podgląd czekających pacjentów)
- **`seed`**: seed generatora liczb pseudolosowych
- **`list_rooms`**: lista gabinetów o długości **`number_of_rooms`**
- **`rooms`**: struktura Store z biblioteki simpy o wielkości **`number_of_rooms`**
- **`processed_patients`**: lista obsłużonych już pacjentów

In [198]:
class Clinic:
    def __init__(self, env, number_of_rooms, service_time, queue_array=[], lambda_per_hour=6/60, seed=None, sim_time=120):
        self.curr_patient_id = 1
        self.env = env
        self.service_time = service_time
        self.lambda_per_hour = lambda_per_hour
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1, env=self.env) for i in range(number_of_rooms)]
        self.rooms = simpy.Store(self.env, capacity=number_of_rooms)
        self.sim_time = sim_time
        self.queue_array = queue_array
        for room in self.list_rooms:
            self.rooms.put(room)
        self.processed_patients = []


### Metoda `czas()`:
Pomocnicza funkcja do ładnego wypisywania czasu.

Parametr `time_in_minutes`: czas w minutach

Praca zaczyna się od godziny 9:00 dlatego hours = 9+[...]

In [199]:

def czas(self, time_in_minutes):
    hours = 9+time_in_minutes//60
    minutes = time_in_minutes%60
    if len(str(int(minutes)))==1:
        return f"{int(hours)}:0{int(minutes)}"
    return f"{int(hours)}:{int(minutes)}"
Clinic.czas = czas

### Metoda `time_between_new_patients()`:
Ważna metoda zwracająca przerwę w minutach, do pojawienia się następnego pacjenta, podawana jest jako parametr metody `env.timeout()` po wygenerowaniu pacjenta.

Wykorzystuje rozkład wykładniczy o intensywności $\frac{1}{\lambda}$

In [200]:
def time_between_new_patients(self):
    if self.seed:
        np.random.seed(self.seed)
    return np.random.exponential(1/self.lambda_per_hour)
Clinic.time_between_new_patients = time_between_new_patients

### Metoda `generate_patients()`:

Metoda wygeneruje pacjenta o jakimś id i rozpocznie dla niego proces `serve_patient()`, aż do momentu {service_time} przed zamknięciem (Uznaliśmy, że pacjenci nie przychodzą np. 15 minut przed zamknięciem). Dzięki niej upłynie czas do przyjścia nastepnego pacjenta

In [201]:
def generate_patients(self):
    while True:
        if self.env.now >= self.sim_time-self.service_time: # Pacjenci nie przychodzą {service_time} przed zamknięciem
            break
        patient = Pacjent(id=self.curr_patient_id)
        patient.arrival_time = self.env.now
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} przybył do kliniki")
        self.env.process(self.serve_patient(patient))
        self.curr_patient_id += 1
        yield self.env.timeout(self.time_between_new_patients())
Clinic.generate_patients = generate_patients

### Metoda `serve_patient(patient)`

Odpalana dla jakiegoś pacjenta:
- Sprawdza czy jakiś pokój jest wolny i dodaje go do podglądu kolejki (queue_array) (Taka prawdziwa kolejka jest wbudowana w SimPy i uruchamiana jest przez `rooms.get()`)
- Jeśli mija godzina *{service_time}* od końca pracy (sim_time) wtedy pacjenci są usuwani z kolejki.
- Obsługuje pacjentów i puszcza czas dalej.

In [202]:
def serve_patient(self, patient): 
    room = yield self.rooms.get()
    self.queue_array.append(patient.id)
    if self.env.now >= self.sim_time-self.service_time: # Nie zdążyło obsłużyć pacjenta przed zamknięciem
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} - nie zdążył zostać obsłużony przed zamknięciem kliniki")
        yield self.rooms.put(room)
        return
    patient.service_start_time = self.env.now
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
    self.queue_array.remove(patient.id)
    yield self.env.timeout(self.service_time)

    patient.service_end_time = self.env.now
    room.patients_served += 1
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wychodzi z gabinetu {room.id}, czekal od {self.czas(np.trunc(patient.arrival_time))} do {self.czas(np.trunc(patient.service_start_time))}")
    patient.room = room.id
    self.processed_patients.append(patient)
    yield self.rooms.put(room)
Clinic.serve_patient = serve_patient

### Metoda `run()` 
Uruchamia symulacje i rozpoczyna proces generowania pacjentów.

In [203]:
def run(self):
    self.env.process(self.generate_patients())
    env.run(until=self.sim_time)
    df = pd.DataFrame([
    {
        "id": p.id,
        "room": p.room,
        "arrival_time": p.arrival_time,
        "service_start_time": p.service_start_time,
        "service_end_time": p.service_end_time,
        "day": p.day
    }
    for p in self.processed_patients
    ])
    return df
Clinic.run = run

In [204]:
def run_multiple_times(self, n):
    for i in range(1, n+1):
        print(f"-----------{i}-----------")
        self.env = simpy.Environment()
Clinic.run_multiple_times = run_multiple_times


## 3. Analiza statystyczna (statystyki opisowe)

In [205]:
def stats(self):
    def patient_bar_plot():
        patients_served_ls = []
        id_ls = []
        for room in self.list_rooms:
            patients_served_ls.append(room.patients_served)
            id_ls.append(str(room.id))
        fig = seaborn.barplot(x=id_ls, y=patients_served_ls)
        fig.set_xlabel('Gabinet')
        fig.set_ylabel('Ilosc Pacjentow')
        plt.savefig("patients_served_noshow.png")
        plt.show()

    patient_bar_plot()
Clinic.stats = stats

In [206]:
#bez umawiania
env = simpy.Environment()

In [207]:
clinic = Clinic(env, number_of_rooms=2, service_time=15)

In [208]:
clinic.run()

Czas 9:00: Pacjent 1 przybył do kliniki
Czas 9:00: Pacjent 1 wchodzi do gabinetu 1 
Czas 9:05: Pacjent 2 przybył do kliniki
Czas 9:05: Pacjent 2 wchodzi do gabinetu 2 
Czas 9:12: Pacjent 3 przybył do kliniki
Czas 9:15: Pacjent 1 wychodzi z gabinetu 1, czekal od 9:00 do 9:00
Czas 9:15: Pacjent 3 wchodzi do gabinetu 1 
Czas 9:15: Pacjent 4 przybył do kliniki
Czas 9:20: Pacjent 2 wychodzi z gabinetu 2, czekal od 9:05 do 9:05
Czas 9:20: Pacjent 4 wchodzi do gabinetu 2 
Czas 9:26: Pacjent 5 przybył do kliniki
Czas 9:29: Pacjent 6 przybył do kliniki
Czas 9:30: Pacjent 3 wychodzi z gabinetu 1, czekal od 9:12 do 9:15
Czas 9:30: Pacjent 5 wchodzi do gabinetu 1 
Czas 9:35: Pacjent 4 wychodzi z gabinetu 2, czekal od 9:15 do 9:20
Czas 9:35: Pacjent 6 wchodzi do gabinetu 2 
Czas 9:43: Pacjent 7 przybył do kliniki
Czas 9:45: Pacjent 5 wychodzi z gabinetu 1, czekal od 9:26 do 9:30
Czas 9:45: Pacjent 7 wchodzi do gabinetu 1 
Czas 9:50: Pacjent 6 wychodzi z gabinetu 2, czekal od 9:29 do 9:35
Czas 9:58:

,id,room,arrival_time,service_start_time,service_end_time,day
0,1,1,0.000000,0.000000,15.000000,1
1,2,2,5.518862,5.518862,20.518862,1
2,3,1,12.427174,15.000000,30.000000,1
3,4,2,15.105202,20.518862,35.518862,1
4,5,1,26.773164,30.000000,45.000000,1
5,6,2,29.669548,35.518862,50.518862,1
6,7,1,43.491169,45.000000,60.000000,1
7,8,2,58.831221,58.831221,73.831221,1
8,9,1,80.744401,80.744401,95.744401,1


In [209]:
clinic.queue_array

[]

In [210]:
clinic.stats()